In [8]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import os
import cv2
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm
from sklearn.preprocessing import MinMaxScaler


In [9]:
scaler = MinMaxScaler()

In [10]:
# Path to the directory containing the image tiles
input_folder = '/home/rubi/Desktop/janakpur/'

# Path to the saved model
model_path = '/home/rubi/Downloads/solidwaste_95.hdf5'
model = tf.keras.models.load_model(model_path,compile=False)


In [11]:
BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

In [12]:
def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize((256, 256)) 
    img_np = np.array(img) / 255.0
    img_np = scaler.fit_transform(img_np.reshape(-1, img_np.shape[-1])).reshape(img_np.shape)
    img_np = preprocess_input(img_np)
    # print(img_np.shape)
    img_np = img_np.reshape((1, 256, 256, 3))

    return img_np

In [13]:
# Function to perform prediction on an image and save the prediction
def predict_and_save(image_path, output_path):
    # Preprocess the image
    img = preprocess_image(image_path)
    # Perform prediction
    prediction = model.predict(img)[0]  # Predict using the pre-trained model
    prediction_img = Image.fromarray((prediction * 255).astype(np.uint8))  # Convert prediction to image format
    prediction_img.save(output_path)

In [14]:
for root, dirs, files in os.walk(input_folder):
    for filename in files:
        if filename.endswith('.png'):  
            tile_path = os.path.join(root, filename)
            # print(filename)
            # Generate output path for prediction (save the prediction in the same folder structure)
            output_folder = '/home/rubi/Desktop/janakpurpred/' 
            output_path = tile_path.replace(input_folder, output_folder)
            
            # Ensure the output directory exists, create if it doesn't
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            
            # # Perform prediction and save the result
            predict_and_save(tile_path, output_path)


1/1 [==============================] - 25s 25s/step


: 